# ライブラリの準備

## 標準的なライブラリ

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px #データの可視化のライブラリ(動作がはやい。)

## 個別の部品

In [ ]:
from sklearn.preprocessing import StandardScaler # 標準化
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
pd.get_option("display.max_columns")
pd.get_option("display.max_rows")

60

# データの読み込み

### おなじみのボストンの住宅価格のデータセット
-  最近のバージョンの`scikit-learn`では`load_boston`は削除済み

In [ ]:
# from sklearn.datasets import load_boston

## 回避
まだ残っているオリジナルのサイトからデータを読み込む

In [ ]:
!wget -q http://lib.stat.cmu.edu/datasets/boston
!head -n 30 boston

 The Boston house-price data of Harrison, D. and Rubinfeld, D.L. 'Hedonic
 prices and the demand for clean air', J. Environ. Economics & Management,
 vol.5, 81-102, 1978.   Used in Belsley, Kuh & Welsch, 'Regression diagnostics
 ...', Wiley, 1980.   N.B. Various transformations are used in the table on
 pages 244-261 of the latter.

 Variables in order:
 CRIM     per capita crime rate by town
 ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
 INDUS    proportion of non-retail business acres per town
 CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
 NOX      nitric oxides concentration (parts per 10 million)
 RM       average number of rooms per dwelling
 AGE      proportion of owner-occupied units built prior to 1940
 DIS      weighted distances to five Boston employment centres
 RAD      index of accessibility to radial highways
 TAX      full-value property-tax rate per $10,000
 PTRATIO  pupil-teacher ratio by town
 B        100

In [ ]:
df = pd.read_csv(
    "http://lib.stat.cmu.edu/datasets/boston_corrected.txt",
    encoding="Windows=1252",
    skiprows=9, # 上位9行は読み飛ばす
    sep="\t"
)
df.head()

,OBS.,TOWN,TOWN#,TRACT,LON,LAT,MEDV,CMEDV,CRIM,ZN,...,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1,Nahant,0,2011,-70.955,42.2550,24.0,24.0,0.00632,18.0,...,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98
1,2,Swampscott,1,2021,-70.950,42.2875,21.6,21.6,0.02731,0.0,...,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14
2,3,Swampscott,1,2022,-70.936,42.2830,34.7,34.7,0.02729,0.0,...,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03
3,4,Marblehead,2,2031,-70.928,42.2930,33.4,33.4,0.03237,0.0,...,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94
4,5,Marblehead,2,2032,-70.922,42.2980,36.2,36.2,0.06905,0.0,...,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33


## データの確認

In [ ]:
features = ["RM", "LSTAT", "DIS", "CRIM"]
target = "MEDV" # 目的変数(家の価格)

In [ ]:
fig = px.scatter_matrix(df, features + [target], size_max=1, opacity=0.2) # opacity(透明度)
fig.update_traces(diagonal_visible=False, showupperhalf=False)
fig.update_layout(width=800, height=800)

In [ ]:
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=123,
)

# 予測モデル

## 関数の準備

### 標準化
DetaFrme を標準化して、DataFrame を返す

In [ ]:
def apply_ss(*x):
    ss = StandardScaler().fit(x[0])
    return tuple(
        pd.DataFrame(ss.transform(xi), columns=xi.columns, index=xi.index)
        for xi in x
    )

### 結果
モデル、データ、評価方法で表を作成

In [ ]:
def get_score(models, **data):
    scores = dict(RMSE=mean_squared_error, R2=r2_score)
    return pd.DataFrame(
        {
            sname: {
                (dname, mname): score(y, model.predict(X))
                for dname, (X, y) in data.items()
                for mname, model in models.items()
            }
            for sname, score in scores.items()
        }
    )

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)
get_score(
    dict(linear=lm),
    train=(X_train, y_train),
    test=(X_test, y_test),
)

,,RMSE,R2
train,linear,26.210898,0.69062
test,linear,40.179792,0.51436


## 線形回帰

### 係数の表示用

In [ ]:
def get_coef(model, name="coef"):
    return pd.Series(
        [model.intercept_] + model.coef_.tolist(), # interceptは切片
        index=["intercept"] + model.feature_names_in_.tolist(),
        name=name,
    )

### そのまま実行

In [ ]:
coef = get_coef(lm)
pd.DataFrame(coef).T

,intercept,RM,LSTAT,DIS,CRIM
coef,-0.19059,5.392303,-0.67681,-0.593671,-0.106654


### LSTAT の2乗を追加

In [ ]:
X_train2 = X_train.copy()
X_test2 = X_test.copy()
X_train2["LSTAT2"] = np.square(X_train2.LSTAT)
X_test2["LSTAT2"] = np.square(X_test2.LSTAT)
X_test2.head()

,RM,LSTAT,DIS,CRIM,LSTAT2
410,5.757,10.11,1.4130,51.13580,102.2121
85,6.630,6.53,4.4377,0.05735,42.6409
280,7.820,3.76,4.6947,0.03578,14.1376
422,5.648,14.10,1.9512,12.04820,198.8100
199,6.975,4.56,7.6534,0.03150,20.7936


In [ ]:
lm2 = LinearRegression()
lm2.fit(X_train2, y_train)
get_score(
    dict(linear2=lm2),
    train=(X_train2, y_train),
    test=(X_test2, y_test),
)

,,RMSE,R2
train,linear2,19.900245,0.765108
test,linear2,31.221687,0.622634


In [ ]:
coef2 = get_coef(lm2, "coef2")
pd.DataFrame(coef2).T

,intercept,RM,LSTAT,DIS,CRIM,LSTAT2
coef2,16.812262,4.296065,-2.068303,-0.852428,-0.147671,0.041155


In [ ]:
pd.DataFrame(
    dict(impact=coef2.LSTAT + 2 * coef2.LSTAT2 * X_test2.LSTAT2),
)

,impact
410,6.344835
85,1.441495
280,-0.904629
422,14.295865
199,-0.356769
...,...
229,-0.904629
159,2.426851
196,-0.698128
345,7.058369


### 標準化

In [ ]:
X_train_ss, X_test_ss = apply_ss(X_train, X_test)
lm_ss = LinearRegression().fit(X_train_ss, y_train)
get_score(
    dict(ss=lm_ss),
    train=(X_train_ss, y_train),
    test=(X_test_ss, y_test),
)

,,RMSE,R2
train,ss,26.210898,0.69062
test,ss,40.179792,0.51436


In [ ]:
coef_ss = get_coef(lm_ss, "coef_ss")
pd.DataFrame([coef, coef_ss])

,intercept,RM,LSTAT,DIS,CRIM
coef,-0.190590,5.392303,-0.676810,-0.593671,-0.106654
coef_ss,22.374752,3.796194,-4.818137,-1.267703,-0.936139


In [ ]:
pd.concat([X_test_ss, y_test], axis=1)

,RM,LSTAT,DIS,CRIM,MEDV
410,-0.745275,-0.381428,-1.111381,5.420728,15.0
85,0.494777,-0.884316,0.305098,-0.398615,26.6
280,2.185112,-1.273421,0.425453,-0.401072,45.4
422,-0.900104,0.179052,-0.859340,0.967497,20.8
199,0.984832,-1.161044,1.811024,-0.401560,34.9
...,...,...,...,...,...
229,0.383982,-1.273421,-0.192522,-0.354817,31.5
159,0.324323,-0.763510,-0.946116,-0.242797,23.3
196,1.428012,-1.228470,1.649740,-0.400579,33.3
345,-0.380220,-0.322430,1.979707,-0.401602,17.5


## RandomForest

In [ ]:
rf = RandomForestRegressor(n_jobs=-1, random_state=123)
rf.fit(X_train, y_train)
get_score(
    dict(Linear=lm, RandomForest=rf),
    train=(X_train, y_train),
    test=(X_test, y_test),
)

RMSE        R2
train Linear        26.210898  0.690620
      RandomForest   1.803551  0.978712
test  Linear        40.179792  0.514360
      RandomForest  23.507811  0.715869